In [ ]:
#%pip install "autogen-ext[magentic-one,openai]" 
#%pip install gradio
# If using the MultimodalWebSurfer, you also need to install playwright dependencies:


## Magentic One Overview

Magentic-One is a generalist multi-agent system for solving open-ended web and file-based tasks across a variety of domains. It represents a significant step forward for multi-agent systems, achieving competitive performance on a number of agentic benchmarks (see the technical report for full details).

When originally released in November 2024 Magentic-One was implemented directly on the autogen-core library. We have now ported Magentic-One to use autogen-agentchat, providing a more modular and easier to use interface.

Using Magentic-One involves interacting with a digital world designed for humans, which carries inherent risks. To minimize these risks, consider the following precautions:

- Use Containers: Run all tasks in docker containers to isolate the agents and prevent direct system attacks.

- Virtual Environment: Use a virtual environment to run the agents and prevent them from accessing sensitive data.

- Monitor Logs: Closely monitor logs during and after execution to detect and mitigate risky behavior.

- Human Oversight: Run the examples with a human in the loop to supervise the agents and prevent unintended consequences.

- Limit Access: Restrict the agents’ access to the internet and other resources to prevent unauthorized actions.

- Safeguard Data: Ensure that the agents do not have access to sensitive data or resources that could be compromised. Do not share sensitive information with the agents. Be aware that agents may occasionally attempt risky actions, such as recruiting humans for help or accepting cookie agreements without human involvement. Always ensure agents are monitored and operate within a controlled environment to prevent unintended consequences. Moreover, be cautious that Magentic-One may be susceptible to prompt injection attacks from webpages.

## Example of using AI Assistant Agents & Magentic One Agents with the MAGENTIC ONE Orchestrator

## Import Required Dependencies

In [ ]:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_ext.agents.file_surfer import FileSurfer
from autogen_ext.agents.magentic_one import MagenticOneCoderAgent
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.teams.magentic_one import MagenticOne
import yaml


## Create a Model Client

In [ ]:
with open("model_config.yaml", "r") as f:
        model_config = yaml.safe_load(f)
model_client = AzureOpenAIChatCompletionClient.load_component(model_config)

## Create Basic Team with one Assistant only

In [ ]:
async def main() -> None:
    assistant = AssistantAgent(
        "Assistant",
        model_client=model_client,
    )
    team = MagenticOneGroupChat([assistant], model_client=model_client)
    await Console(team.run_stream(task="Provide proof of the Pythagorean theorem using a diagram."))
    await model_client.close()


await main()


## Create a team with a multimodal websurfer

- WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.

In [ ]:
# The example code may download files from the internet, execute code, and interact with web pages. 
# Ensure you are in a safe environment before running the example code.
async def main() -> None:
    surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=model_client,
    )

    team = MagenticOneGroupChat([surfer], model_client=model_client)
    await Console(team.run_stream(task="What is the UV index in Toronto today?"))




await main()


## Create a multi-agent team with the following capabilities

- WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.

- FileSurfer: An agent that can handle local files.

- Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.

- ComputerTerminal: A computer terminal that performs no other action than running Python scripts (provided to it quoted in python code blocks), or sh shell scripts (provided to it quoted in sh code blocks).



In [3]:

# The example code may download files from the internet, execute code, and interact with web pages. 
# Ensure you are in a safe environment before running the example code.

async def main() -> None:
    surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=model_client,
    )

    file_surfer = FileSurfer( "FileSurfer",model_client=model_client)
    coder = MagenticOneCoderAgent("Coder",model_client=model_client)
    terminal = CodeExecutorAgent("ComputerTerminal",code_executor=LocalCommandLineCodeExecutor())
    
    team = MagenticOneGroupChat([surfer, file_surfer, coder, terminal], model_client=model_client)

    await Console(team.run_stream(task="What is the UV index in Melbourne today?"))



await main()


/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:164: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:177: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (user) ----------
What is the UV index in Melbourne today?
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

What is the UV index in Melbourne today?


To answer this request we have assembled the following team:

WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.
FileSurfer: An agent that can handle local files.
Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.
ComputerTerminal: A computer terminal that performs no other action than running Python scripts (provide

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:304: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:610: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- MultiModalMessage (WebSurfer) ----------
I typed 'current UV index in Melbourne today' into '0 characters out of 2000'.

The web browser is open to the page [current UV index in Melbourne today - Search](https://www.bing.com/search?q=current+UV+index+in+Melbourne+today&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=3E46FA61C87049C68AF848B047255284).
The viewport shows 31% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

By using this site you agree to the use of cookies for analytics, personalized content, and ads.
Learn more about third party cookies
|
Microsoft Privacy Policy 
Skip to content
current UV index in Melbourne todayFrançaisAll
Search
Copilot
News
Images
Videos
Maps
More
Tools
About 515,000 resultsMelbourne, Victoria, AustraliaUpdated a few minutes ago
Share
Print
Feedback
14
°C
F18°12°
Wind: 8 KMPH
Humidity: 89%
Partly cloudy
Tue 29, 6:58 a.m.
7 AM
10 AM
1 PM
4 PM
7 PM
10 PM
1 AM
4 AM
1 AM
4 AM
7 AM
10 AM
1

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:304: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)


---------- MultiModalMessage (WebSurfer) ----------
I clicked 'WillyWeather'.

The web browser is open to the page [Melbourne UV Index, VIC 3000 - WillyWeather](https://uv.willyweather.com.au/vic/melbourne/melbourne.html).
The viewport shows 41% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

75,825
0
6:58 am 
29 AprNow
0 Now 
Low
Max UV Today
3.7 Moderate
Moderate risk of harm from unprotected Sun exposure. Alert in effect for 
3 hours between 
11am–2pm.Weather
Wind
Rainfall
Sun
Moon
UV
Tides
Swell
More
Melbourne 
 UV IndexAustraliaVICMelbourne1-Day
3-Day
UV Index
Tue 29 AprWed 30 AprThu 1 MayFri 2 MaySat 3 May3+ Moderate
6+ High
8+ Very High
11+ Extreme

The following metadata was extracted from the webpage:

{
    "meta_tags": {
        "apple-itunes-app": "app-id=592978502, app-argument=https://itunes.apple.com/au/app/weather-by-willyweather/id592978502?mt=8&uo=4&at=11lMKC%22",
        "og:url": "https://uv.willyweather.com.a

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:455: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


## This uses Magentic_one helper class with all the agents bundled together



In [ ]:



async def example_usage():
    client = model_client
    m1 = MagenticOne(client=client)
    task = "Write a Python script to fetch data from the OpenWeatherMap API, and save the file locally."
    result = await Console(m1.run_stream(task=task))
    print(result)


if __name__ == "__main__":
    await example_usage()
